# Break it down
1. Decide what kind of data you want the application to manage. Examples include budget items, work request tickets, investment performance, book reviews, and more.
2. Create the .csv file with specific columns and a first row or two as "seed" data for your application to test while developing.
3. Create your index route and template that shows all items in the .csv file on the web page, including links to each individual item's details page.
4. Complete the show route that displays the details of each item in the .csv
5. Include a create route that allows users to add new data to the file. This should accept a POST request with the new data.


## Prelim

In [ ]:
# Decide what kind of data you want the application to manage. 
# Examples include budget items, work request tickets, investment performance, 
# book reviews, and more.

import pandas as pd
data = pd.read_csv('./international_football_results.csv')
data.head()

In [ ]:
# Create the .csv file with specific columns and a 
# first row or two as "seed" data for your application to test while developing.

data['year'] = pd.to_datetime(data['date']).dt.year
data.head()

In [ ]:
data.sample(10).to_csv('seed.csv',index=False)

In [ ]:
import pandas as pd
seed = pd.read_csv('seed.csv')
seed.head()

In [ ]:
seed['key'] = seed['home_team']+seed['away_team']+seed['date']
seed.to_csv('seed.csv',index=False)

## Flask

In [1]:
# Create your index route and template that shows all items in the 
# .csv file on the web page, including links to each individual 
# item's details page.

# setup
from flask import Flask, request, render_template
import csv

app = Flask(__name__) # invoke the Flask class

In [2]:
# step 1 - index route
@app.route('/index')
def index():
    data_list = []
    with open('seed.csv', mode='r') as file: 
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    # Pass the list of dictionaries to the template
    return render_template('index.html', data_list=data_list)

In [3]:
# step 2 - show route
@app.route('/show/<key>')
def show(key):
    data_list = []
    with open('seed.csv', mode='r') as file: 
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    for data in data_list:
        if data['key'] == key:
            print('True')
            date = data['date']
            home_team = data['home_team']
            
    return render_template('show.html', date = date, home_team = home_team)

In [4]:
# step 3 - data post route
@app.route('/data')
def data():
    """Designed for accepting posts as an API request"""
    if request.method == 'POST':
        form_data = request.form
        # Open the CSV file in append mode and write the form data
        with open('seed.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            # Assuming 'name' and 'greeting' are the keys in form_data dictionary
             
            writer.writerow([
                form_data['date'], 
                form_data['home_team'],
                form_data['away_team'],
                form_data['home_score'],
                form_data['away_score'],
                form_data['tournament'],
                form_data['city'],
                form_data['country'],
                form_data['neutral'],
                form_data['year'],
                form_data['home_team']+form_data['away_team']+form_data['date'],
            ])
        # Redirect or show confirmation to the user
        return {'status_code':'200'}

In [ ]:
# run the server, test, and see! url is http://127.0.0.1:9001/
if __name__ == '__main__':
    app.run(port=9001) # Start the server listening for requests

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Index</title>
</head>
<body>
    <h1>Soccer Games</h1>
    <ul>
        {% for data in data_list %}
        <li><a href="/show/{{data['key']}}">{{ data['tornament'] }}: {{ data['date'] }}</a></li>
        {% endfor %}
    </ul>

    <a href="/form">Add a new greeting!</a>
</body>
</html>
```
Example
```python
{'date': '2005-10-08',
  'home_team': 'Uganda',
  'away_team': 'Burkina Faso',
  'home_score': '2',
  'away_score': '2',
  'tournament': 'FIFA World Cup qualification',
  'city': 'Kampala',
  'country': 'Uganda',
  'neutral': 'False',
  'year': '2005'},
```